# Setup

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account).
2. (Optional) If you would like to do the deep learning component of this tutorial, turn on the GPU with Edit->Notebook settings->Hardware accelerator->GPU


Julia and Julia dependencies are installed at first import:

In [1]:
import pysr

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


Now, let's import everything else as well as the PySRRegressor:


In [2]:
import sympy
import numpy as np
from matplotlib import pyplot as plt
from pysr import PySRRegressor
from sklearn.model_selection import train_test_split

# Simple PySR example:


First, let's learn a simple function

$$2.5382 \cos(x3) + x0^2 - 2$$

In [3]:
from scipy.integrate import quad

In [4]:
def _matsubara_zero_integrand(t, coup_strength, bath_broad, bath_freq):
    """
    Integral for the zero temperature Matsubara exponentials.
    """
    lam = coup_strength
    gamma = bath_broad
    w0 = bath_freq

    omega = np.sqrt(w0 ** 2 - (gamma / 2) ** 2 +0j )
    a = omega + 1j * gamma/2 
    aa = np.conjugate(a)

    prefactor = -(lam ** 2 * gamma) / np.pi
    integrand = lambda x: prefactor * ((x * np.exp(-x * t)) / ((a ** 2 + x ** 2) * (aa ** 2 + x ** 2)))
    return quad(integrand, 0, np.inf,limit=5000,complex_func=True)[0]

In [5]:
funcx=np.vectorize(_matsubara_zero_integrand)

In [6]:
def matsubara_zero_integrand(X):
    """
    Integral for the zero temperature Matsubara exponentials.
    """
    lam = np.ones(len(X[:,0]))
    gamma = X[:,0]
    w0 = X[:,1]
    t=X[:,2]
    return funcx(t,lam,gamma,w0)

In [53]:
X0 = 5 *np.abs(np.random.randn(100000, 1))
X1 = 10*(np.abs(np.random.randn(100000, 1))+1)
X2 = 10*np.abs(np.random.randn(100000, 1))
X=np.hstack([X0,X1,X2])

In [54]:
(2*X[:,1] >X[:,0]).all()

True

In [55]:
y=matsubara_zero_integrand(X+0j)
y.shape

(100000,)

By default, we will set up 30 populations of expressions (which evolve independently except for migrations), use 4 threads, and use `"best"` for our model selection strategy:

In [56]:
default_pysr_params = dict(
    populations=10,
    model_selection="best",
)

PySR can run for arbitrarily long, and continue to find more and more accurate expressions. You can set the total number of cycles of evolution with `niterations`, although there are also a [few more ways](https://github.com/MilesCranmer/PySR/pull/134) to stop execution.

**This first execution will take a bit longer to startup, as the library is JIT-compiled. The next execution will be much faster.**

In [57]:
# Learn equations
model = PySRRegressor(
    niterations=50,
    binary_operators=["+", "*","-","/"],
    unary_operators=["exp"],
    **default_pysr_params,
    procs=10,
)

model.fit(X, y)

We can print the model, which will print out all the discovered expressions:

In [47]:
model

PySRRegressor.equations_ = [
	    pick     score                                           equation  \
	0         0.000000                                          0.6926408   
	1         4.364914                                     exp(-2.561197)   
	2         7.108481                                x0 * -0.00041184324   
	3         0.041814                            -0.0022641895 / exp(x2)   
	4         0.141244                 -0.00041184344 / (x2 + 0.07887375)   
	5         0.147830                       -0.0076869363 / exp(x1 * x2)   
	6         0.310864           -0.0002122705 / ((x2 + 0.07572268) / x0)   
	7         0.105444             x0 / (exp(x1 + exp(x2)) * -0.45525673)   
	8         0.058640            (x0 / exp(x2 + x1)) / (-0.9219685 - x2)   
	9         0.012243             ((x2 - x0) / exp(x1)) / (exp(x2) + x2)   
	10        0.140949  (x0 / exp(-0.00041184324 + x1)) / (-0.9219685 ...   
	11        0.471422  (-0.28290802 * exp(((-0.28290802 + 0.60634625)...   
	12  >>>>  0.443509  (x2 - x0) * (exp(exp(x2) * (-0.60770625 * x1))...   
	13        0.053995  ((x0 / x1) - x0) * (exp(exp(x2) * (x1 * -0.607...   
	14        0.003257  ((x0 / (exp(x2) + x1)) - x0) * (exp(exp(x2) * ...   
	
	            loss  complexity  
	0   4.810965e-01           1  
	1   6.117499e-03           2  
	2   5.004953e-06           3  
	3   4.799993e-06           4  
	4   4.167726e-06           5  
	5   3.594986e-06           6  
	6   2.634455e-06           7  
	7   2.133551e-06           9  
	8   2.012037e-06          10  
	9   1.987553e-06          11  
	10  1.726257e-06          12  
	11  1.077382e-06          13  
	12  6.914428e-07          14  
	13  6.206641e-07          16  
	14  6.146295e-07          19  
]

We can also view the SymPy format of the best expression:

In [49]:
model.sympy().simplify()

0.17934802267784*(-x0 + x2)*exp(-0.60770625*x1*exp(x2))

We can also view the SymPy of any other expression in the list, using the index of it in `model.equations_`.

In [52]:
model.sympy()

(-x0 + x2)*exp(-1.7184271)*exp(exp(x2)*(-0.60770625)*x1)

## Output

`model.equations_` is a Pandas DataFrame. We can export the results in various ways:

In [15]:
model.latex()

'- \\frac{1.47 x_{0}}{e^{x_{1}} e^{x_{1} x_{2}}}'

These is also `model.sympy(), model.jax(), model.pytorch()`. All of these can take an index as input, to get the result for an arbitrary equation in the list.

We can also use `model.predict` for arbitrary equations, with the default equation being the one chosen by `model_selection`:

In [16]:
ypredict = model.predict(X)
ypredict_simpler = model.predict(X, 2)

print("Default selection MSE:", np.power(ypredict - y, 2).mean())
print("Manual selection MSE for index 2:", np.power(ypredict_simpler - y, 2).mean())

Default selection MSE: (1.540410901824946e-06+0j)
Manual selection MSE for index 2: (4.909795905743584e-06+0j)


In [32]:
model.predict(np.array([[1,1.2,1]]))

array([-0.13357556])

In [34]:
_matsubara_zero_integrand(1,1,1,1.2)

(-0.04096309938986264+0j)

In [35]:
matsubara_zero_integrand(np.array([[1,1.2,1]]))[0]

(-0.04096309938986264+0j)